In [ ]:
!pip install streamlit

In [ ]:
!pip install easyocr

In [ ]:
!pip install pytesseract

In [ ]:
!pip install opencv-python

In [ ]:
!pip install pyngrok
     

In [13]:
! ngrok config add-authtoken 2P0LznujREuShZx0WUKVWjMFlpN_2z2P1e8CFMWZ6Sjz32TJP

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [15]:
!./ngrok authtokens 2P0LznujREuShZx0WUKVWjMFlpN_2z2P1e8CFMWZ6Sjz32TJP

/bin/bash: ./ngrok: No such file or directory


In [9]:
from pyngrok import ngrok 

In [40]:
%%writefile bizxapp.py

import streamlit as st
import easyocr
import sqlite3
import cv2
import numpy as np 
import pandas as pd


# Creating connection sqlite3 database
conn = sqlite3.connect('card_extraction.db')
c = conn.cursor()


# Creating a table to store the business card information
c.execute('''CREATE TABLE IF NOT EXISTS card_data 
              (id INT AUTO_INCREMENT PRIMARY KEY,
              name TEXT,
              position TEXT,
              address TEXT,
              pincode VARCHAR(25),
              phone VARCHAR(25),
              email TEXT,
              website TEXT,
              company TEXT
              )''')

#EasyOCR to read the data
reader = easyocr.Reader(['en'])

# Title 
st.title(":red[Business card extraction using OCR]")

# File uploader
file_upload = st.file_uploader(":green[UPLOAD CARD]", type=["jpg", "jpeg", "png", "tiff", "tif", "gif"])


# Creating side bar
st.sidebar.title(":purple[BIZCARDX]")

tabs = ['Insert Data', 'View Data', 'Edit Card', "Delete card"]
choose = st.sidebar.selectbox("Select", tabs)

if choose == 'Insert Data':
    if file_upload is not None:
        
        # Read the image using OpenCV
        image = cv2.imdecode(np.fromstring(file_upload.read(), np.uint8), 1)
        
        # Display the uploaded img
        st.image(image, caption='Ready for Extraction', use_column_width=True)
        
        # Extracting information from the image
        if st.button('Extract Data'):
            bsc = reader.readtext(image, detail=0)
            text = "\n".join(bsc)
            
            # Extracted data to be stored in sqliteDB
            sql_data = "INSERT INTO card_data (name, position, address, pincode, phone, email, website, company) VALUES ( ?, ?, ?, ?, ?, ?, ?, ?)"
            values = (bsc[0], bsc[1], bsc[2], bsc[3], bsc[4], bsc[5], bsc[6], bsc[7])
            c.execute(sql_data, values)
            conn.commit()

            st.success("card Extracted stored on View Data Tab")

elif choose == 'View Data':
    
    # show Extracted business card information
    c.execute("SELECT * FROM card_data")
    result = c.fetchall()
    df = pd.DataFrame(result, columns=['id','name', 'position', 'address', 'pincode', 'phone', 'email', 'website', 'company'])
    st.write(df)


elif choose == 'Edit Card':
    
    # Create a dropdown menu to select a business card to edit
    c.execute("SELECT id, name FROM card_data")
    result = c.fetchall()
    business_cards = {}
    
    for row in result:
        business_cards[row[1]] = row[0]
    select_card_name = st.selectbox("Select Card To Edit", list(business_cards.keys()))
    
    # Getting the information of selected business card
    c.execute("SELECT * FROM card_data WHERE name=?", (select_card_name,))
    result = c.fetchone()

    # edited information 
    name = st.text_input("Name", result[1])
    position = st.text_input("Position", result[2])
    address = st.text_input("Address", result[3])
    pincode = st.text_input("Pincode", result[4])
    phone = st.text_input("Phone", result[5])
    email = st.text_input("Email", result[6])
    website = st.text_input("Website", result[7])
    company = st.text_input("Company_Name", result[8])

    
    #Button for to updating the business card
    if st.button("Edit Card"):
        
        # Update the information for the selected business card in the DB
        c.execute("UPDATE card_data SET name=?, position=?, address=?, pincode=?, phone=?, email=?, website=?, company=? WHERE name=?", 
                             (name, position, address, pincode, phone, email, website, company, select_card_name))
        conn.commit()
        st.success("Bizcard Updated")

if choose == 'Delete card':
    
    # Creating  dropdown menu to delete the stored Business card
    c.execute("SELECT id, name FROM card_data")
    result = c.fetchall()
    business_cards = {}
    
    for row in result:
        business_cards[row[1]] = row[0]
    select_card_name = st.selectbox("Select Card To Delete", list(business_cards.keys()))
    
    # Button for delete the card
    if st.button("Delete Card"):
        
        # Deleting the info on DB
        c.execute("DELETE FROM card_data WHERE name=?", (select_card_name,))
        conn.commit()
        st.success("Card Deleted")


Writing bizxapp.py


In [ ]:
!streamlit run /content/bizxapp.py & npx localtunnel -p 8501